In [ ]:
USE_PRIVATE_DISTRO = True

DRIVE_BASE_DIR = '/content/drive/MyDrive/SMC 10/DDSP-10/'

if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU /content/drive/MyDrive/SMC\ 10/DDSP-10/dist/ddsp-1.2.0.tar.gz
else:
    !pip install -qU ddsp

#Import libraries

In [ ]:
import keras
from keras import layers
from keras import backend
from keras import losses

import tensorflow as tf

import numpy as np

from ddsp import synths
from ddsp.colab import colab_utils

import seaborn as sns
from matplotlib import pyplot as plt
sns.set(style="whitegrid")
%config InlineBackend.figure_format='retina'

# Generate synthetic dataset

In [ ]:
SAMPLE_RATE = 48000
DURATION = 4

N_SAMPLES = SAMPLE_RATE * DURATION

fm_synth = synths.FrequencyModulation(n_samples=N_SAMPLES,
                                      sample_rate=SAMPLE_RATE,
                                      amp_scale_fn=None)

algorithms = [[1,0,0,0], [1,0,0,0], [1,0,0,0], 
              [1,1,0,0], [1,1,0,0], [1,1,0,0], 
              [1,1,0,0], [1,1,1,0], [1,1,1,0], 
              [1,1,1,0], [1,1,1,1], [1,0,0,0], 
              [1,0,0,0]] 

connections=[[43,32,21],[42,32,21],[21,31,43],
            [32,43],[31,42],[32,42],
            [31,32,43],[41],[41,42],
            [41,42,43],[],[21,31,41],
            [21,31,42,43]]

mods = {'21':0, '31':0, '32':0, '41':0, '42':0, '43':0}

In [ ]:
DATA_SIZE = 100000
TRAIN_SIZE = 8*DATA_SIZE//10
TEST_SIZE = DATA_SIZE - TRAIN_SIZE

patch_x = []
patch_y = []

for patch in range(DATA_SIZE):

  current_patch=[]
  current_algorithm = np.random.randint(13)

  #Operators
  for f in range(4):
    amp = float(algorithms[current_algorithm][f])
    current_patch.append(amp)
    index = 1.0/4.5 if f==0 else np.random.randint(1,10)*0.5/4.5
    current_patch.append(index)
    current_patch += np.random.random(2).tolist()

  #Modulators
  for modulator in mods.keys():
    if int(modulator) in connections[current_algorithm]:
      mods[modulator] = float(np.random.randint(0,12))
    else:
      mods[modulator] = 0

  current_patch += [mods['21']/11.0, mods['31']/11.0, mods['32']/11.0, mods['41']/11.0, mods['42']/11.0, mods['43']/11.0]

  patch_x.append(current_patch)
  patch_y.append(current_algorithm)

patch_x = np.array(patch_x)
patch_y = np.array(patch_y)

x_train = patch_x[:TRAIN_SIZE,:]
y_train = patch_y[:TRAIN_SIZE]

x_test = patch_x[-TEST_SIZE:,:]
y_test = patch_y[-TEST_SIZE:]

#Define autoencoder

In [ ]:
USE_VAE = True

LATENT_DIMS = 2
PARAMETERS = 22

UNITS_HIDDEN_LAYER = 64

In [ ]:
def sampling(args):
  z_mean, z_log_sigma = args
  epsilon = backend.random_normal(shape=backend.shape(z_mean), stddev=0.1)
  return z_mean + backend.exp(0.5 * z_log_sigma) * epsilon

In [ ]:
#common for both models
input_encoder = keras.Input(shape=(PARAMETERS,), name='input_encoder')
encoder = layers.Dense(UNITS_HIDDEN_LAYER, activation='relu')(input_encoder)

if USE_VAE:
  z_mean = layers.Dense(LATENT_DIMS, name='mean')(encoder)
  z_log_sigma = layers.Dense(LATENT_DIMS, name='log_sigma')(encoder)
  z = layers.Lambda(sampling, name='z')([z_mean, z_log_sigma])
  encoder_nn = keras.Model(input_encoder, [z_mean, z_log_sigma, z], name='encoder')
else:
  encoder = layers.Dense(LATENT_DIMS, activation='relu', name='z')(encoder)
  encoder_nn = keras.Model(input_encoder, encoder, name='encoder')

#common for both models
input_decoder=keras.Input(shape=(LATENT_DIMS,), name='input_decoder')
decoder = layers.Dense(UNITS_HIDDEN_LAYER, activation='relu')(input_decoder)
decoder = layers.Dense(PARAMETERS, activation='sigmoid', name='decoder_output')(decoder)
decoder_nn = keras.Model(input_decoder, decoder, name='decoder')

if USE_VAE:
  output_decoder = decoder_nn(encoder_nn(input_encoder)[2])
else:
  output_decoder = decoder_nn(encoder_nn(input_encoder))

#common for both models
autoencoder = keras.Model(input_encoder,output_decoder, name='autoencoder')

reconstruction_loss = losses.binary_crossentropy(input_encoder, output_decoder)
# reconstruction_loss = losses.mse(input_encoder, output_decoder)
reconstruction_loss *= PARAMETERS
        
if USE_VAE:
  kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
  kl_loss = backend.sum(kl_loss, axis=-1)
  kl_loss *= -0.5
  loss = backend.mean(reconstruction_loss + kl_loss)
else:
  loss = reconstruction_loss

autoencoder.add_loss(loss)
autoencoder.compile(optimizer='adam')

# autoencoder.summary()
# encoder_nn.summary()
# decoder_nn.summary()

#Train autoencoder

In [ ]:
_ = autoencoder.fit(x_train, x_train,
                epochs=300,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

#Analyse results

In [ ]:
if USE_VAE:
  mean, log_sigma, encoded_data = encoder_nn.predict(x_test)
else:
  encoded_data = encoder_nn.predict(x_test)
  
decoded_data = decoder_nn.predict(encoded_data)

In [ ]:
element = np.random.randint(x_test.shape[0])
for f in range(PARAMETERS):
  if f in [0,4,8,12]:
    print("op {}".format(1+f//4))
  if f==16:
    print("mods")
  print('\t{:.2f}\t{:.2f}'.format(x_test[element][f],decoded_data[element][f]))

In [ ]:
x_from = -1
x_to = 1

y_from = 2
y_to = -2

# decoder_nn.predict([[x_from,y_from], [x_to,y_to]])
samples = np.linspace(start=[x_from,y_from], stop=[x_to,y_to], num=1500)
# samples
predicted = decoder_nn.predict(samples)

In [ ]:
plt.figure(figsize=(12, 6))
# plt.gca().set_aspect('equal', adjustable='box')
plt.set_cmap('jet')
plt.scatter(encoded_data[:,0], encoded_data[:,1], c=y_test, s=1, alpha=0.6)
plt.scatter([x_from,x_to],[y_from,y_to], c='white', s=100, edgecolors='black')

plt.colorbar()
plt.show()

f, ax = plt.subplots(2, 2, figsize=(12, 6), sharex=True)

ax[0][0].set_title('Amp')
for f in range(4):
  ax[0][0].plot(predicted[:,f*4])

ax[0][1].set_title('Index')
for f in range(4):
  ax[0][1].plot(predicted[:,f*4+1])

ax[1][0].set_title('delta AR')
for f in range(4):
  ax[1][0].plot(predicted[:,f*4+2]-predicted[:,f*4+3])

ax[1][1].set_title('Modulators')
for f in range(6):
  ax[1][1].plot(predicted[:,16+f])
plt.show()

In [ ]:
if USE_VAE:
  plt.figure(figsize=(12, 6))
  plt.set_cmap('jet')
  plt.scatter(mean[:,0], mean[:,1], c=y_test, s=2, alpha=0.6)
  plt.colorbar()
  plt.show()

In [ ]:
raise Exception("Stop right now!!!") 

#Save full model

In [ ]:
modelname = 'vae_params_std01' #@param {type: "string"}

In [ ]:
autoencoder.save(DRIVE_BASE_DIR + 'models/' + modelname)